<a href="https://colab.research.google.com/github/seonmia/MRC_KoBigBird_KoELECTRA/blob/main/preprocessing_answer_length.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## data_extraction() 메소드
json 훈련 데이터 파일에서 answer길이가 긴것들 제거하는 함수

* 첫 번째 인자 : json 파일 경로
* 두 번째 인자 : 허용가능한 answer의 길이

In [ ]:
import json
from pprint import pprint

def data_extraction(json_file_path: str, answer_max_length: int):
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    data_cut_indices = []
    print("수정 전 데이터 개수", len(data['data']))
    
    # print("<<<<<짧은 답만 남긴 데이터들>>>>>")

    for i in range(len(data['data'])):
        for j in range(len((data['data'][i]['paragraphs']))):
            for m in range(len(data['data'][i]['paragraphs'][j]['qas'])):
                min_answer_length = len(data['data'][i]['paragraphs'][j]['qas'][m]['answers'][0]['text'])
                min_answer_index = 0
                for n in range(len(data['data'][i]['paragraphs'][j]['qas'][m]['answers'])):
                    if min_answer_length > len(data['data'][i]['paragraphs'][j]['qas'][m]['answers'][n]['text']):
                        min_answer_length = len(data['data'][i]['paragraphs'][j]['qas'][m]['answers'][n]['text'])
                        min_answer_index = n
                # answer들이 모두다 answer_max_length 넘을때 바로 삭제할 데이터의 인덱스를 저장 => 뒤에서 삭제
                if min_answer_length > answer_max_length:
                    data_cut_indices.append((i,j))
                # answer가 1개인 경우, 만약 길이가 answer_max_length 넘는다면 위에서 추출될테니 여기의 answer는 answer_max_length 이하이다.
                # 따라서 따로 작업을 하지 않는다.
                elif len(data['data'][i]['paragraphs'][j]['qas'][m]['answers']) == 1:
                    continue
                # answer가 여러개이고 최소한 1개는 answer_max_length 를 넘지않는다.
                # 그 중 제일 짧은 답 하나만 남기도록 한다.
                else:
                    print("=====Before answers:=====")
                    pprint(data['data'][i]['paragraphs'][j]['qas'][m]['answers'])
                    
                    answer = data['data'][i]['paragraphs'][j]['qas'][m]['answers'][min_answer_index]
                    del data['data'][i]['paragraphs'][j]['qas'][m]['answers']
                    data['data'][i]['paragraphs'][j]['qas'][m]['answers'] = [answer]
                    
                    print("=====After answers:=====")
                    pprint(data['data'][i]['paragraphs'][j]['qas'][m]['answers'])

    # print("<<<<<완전히 삭제한 데이터들>>>>>")
    for i, j in reversed(data_cut_indices):
        # pprint(data['data'][i]['paragraphs'][j]) 
        del data['data'][i] # index 에러로 수정

    print("삭제된 데이터 개수", len(data_cut_indices))
    print("수정 후 데이터 개수", len(data['data']))

    return data

## 사용 예시

In [ ]:
answer_max_length = 20
data = data_extraction('/content/TL_span_extraction5.json', answer_max_length)

수정 전 데이터 개수 111967
삭제된 데이터 개수 12594
수정 후 데이터 개수 99373


## 제거되고 남은 데이터를 저장

In [ ]:
with open(f'data_extraction_answer_len{answer_max_length}.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

## PC로 다운로드

In [ ]:
from google.colab import files
files.download(f'data_extraction_answer_len{answer_max_length}.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 공유문서 저장